In [1]:
!pip install minsearch


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [4]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [5]:
question = 'Can I still join the course?'

In [6]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [7]:
search_results = search(question)

In [8]:
prompt = build_prompt(question, search_results)

In [9]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT>
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.

section: General

In [11]:
from openai import OpenAI
import yaml
# Open the file
with open('../api_keys.yml', 'r') as file:
    # Load the data from the file
    data = yaml.safe_load(file)
    
# Get the API key
openai_api_key = data['OPENAI_API_KEY']

client = OpenAI(api_key=openai_api_key)

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [11]:
answer = llm(prompt)

In [12]:
print(answer)

Yes, you can still join the course even after the start date. You are eligible to submit homework assignments regardless of whether you have registered. However, be mindful that there will be deadlines for turning in the final projects, so it's important not to leave everything until the last minute.


In [13]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
rag("How do I patch KDE under FreeBSD?")

"I'm sorry, but the context provided does not contain any information on how to patch KDE under FreeBSD. Please provide additional details or context for me to assist you further."

In [15]:
print(llm("How do I patch KDE under FreeBSD?"))

Patching KDE under FreeBSD involves a few steps, depending on whether you want to apply a security patch, make an enhancement, or fix a bug. Here’s a general guide on how to patch KDE on FreeBSD:

### Step 1: Install Ports Collection

Before you can patch KDE, ensure you have the FreeBSD ports tree installed. You can do this with the following commands:

```sh
portsnap fetch update
```

### Step 2: Locate the KDE Port

Identify which KDE component you want to patch. You can find KDE ports in the `/usr/ports/x11/kde` directory or similar paths. Use the `ls` command to list available ports:

```sh
cd /usr/ports/x11/kde
ls
```

### Step 3: Download the Patch

Get the patch file. This can be done through several ways depending on the nature of the patch:
- Download it manually from a bug tracker or mailing list.
- Use `fetch` if you have a URL.

If you have a patch file named `fix-bug.patch`, you can download it directly or receive it from a source control website.

### Step 4: Apply the P

In [16]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [17]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [18]:
prompt = prompt_template.format(question=question, context=context)

In [19]:
answer_json = llm(prompt)

In [20]:
import json

In [21]:
answer = json.loads(answer_json)

In [22]:
answer['action']

'SEARCH'

In [23]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [24]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)

In [25]:
answer_json = llm(prompt)

In [26]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course after the start date. Even if you do not officially register, you are eligible to submit homework assignments. However, make sure to adhere to deadlines for final projects.",
"source": "CONTEXT"
}


In [27]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [28]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [29]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [30]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [31]:
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number

In [47]:
answer_json = llm(prompt)

In [49]:
answer = json.loads(answer_json)

In [41]:
previous_actions.append(answer)

In [42]:
keywords = answer['keywords']

In [43]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [44]:
search_results = dedup(search_results)

In [46]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [51]:
answer['answer']

"To do well in Module 1, which focuses on Docker and Terraform, here are some general tips: \n1. **Understand the Basics**: Make sure you have a strong grasp of Docker and Terraform concepts, commands, and configurations. \n2. **Practice Regularly**: Set up your own Docker containers and practice writing Terraform scripts to cement your understanding. \n3. **Follow Documentation**: Refer to the official documentation for Docker and Terraform for the most accurate and detailed information. \n4. **Engage in Practical Assignments**: Complete all assigned projects and exercises thoroughly, as practical application reinforces learning. \n5. **Join Study Groups**: Collaborate with peers to discuss module topics and troubleshoot problems together. \n6. **Ask Questions**: Don’t hesitate to reach out for help if you're stuck or confused about certain concepts. \n7. **Utilize Resources**: Make use of additional learning resources like online tutorials, videos, and forums to broaden your understa

In [52]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = json.loads(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

In [53]:
answer

{'action': 'ANSWER',
 'answer': "To be successful in Module 1, which focuses on Docker and Terraform, here are some key tips: \n\n1. **Understand Core Concepts**: Make sure you have a solid understanding of basic concepts related to Docker and Terraform. This includes how containers work, the purpose of Docker images, and how to manage infrastructure as code using Terraform.\n\n2. **Hands-On Practice**: Engage in practical exercises. Set up your own Docker environment, create Dockerfiles, and use Terraform to manage resources. Implement real-world scenarios to solidify your understanding.\n\n3. **Utilize Resources**: Leverage provided course resources, including any reading materials, documentation, and forums. Docker and Terraform have extensive documentation that can deepen your knowledge.\n\n4. **Regular Review**: Continually review learning materials and your notes to reinforce the concepts over time.\n\n5. **Seek Help When Needed**: Don't hesitate to ask questions in discussion fo

In [54]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [55]:
search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [56]:
question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = [search_tool]

chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

response = client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=tools
)
response.output

[ResponseFunctionToolCall(arguments='{"query":"how to do well in module 1"}', call_id='call_HcsfhOElxNVp1udtTda8xu6T', name='search', type='function_call', id='fc_6876cf1e0058819a866d45663784b8b80da3b9f03fc39b9e', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"module 1 tips for success"}', call_id='call_mjvlPuBHscXbp5dZjUD9Lwo0', name='search', type='function_call', id='fc_6876cf1e7914819ab02fa73b5bfe8c310da3b9f03fc39b9e', status='completed'),
 ResponseFunctionToolCall(arguments='{"query":"strategies for excelling in module 1"}', call_id='call_Tw5hcHtT8nXb28zezz79bVU0', name='search', type='function_call', id='fc_6876cf1ed1c4819a8fad25f89e05dd2c0da3b9f03fc39b9e', status='completed')]

In [2]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [17]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data from the weather DB",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Check the weather in the city in question"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [6]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-15 22:47:39--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3.40K  --.-KB/s    in 0s      

2025-07-15 22:47:39 (79.8 MB/s) - ‘chat_assistant.py’ saved [3485/3485]



In [7]:
import chat_assistant

In [18]:
tools = chat_assistant.Tools()

tools.add_tool(get_weather,get_weather_tool)

In [19]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data from the weather DB',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Check the weather in the city in question'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [20]:
developer_prompt = """
You're given a question from a user about the weather and your task is to answer it.

Determine the most appropriate city from the user's question and use known_weather_data if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [21]:
chat.run()

You: What's the weather in Berlin?


You: stop


Chat ended.


In [16]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_description = {
    "type": "function",
    "name": "set_weather",
    "description": "Add an entry to the weather database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to be added to the weather database",
            },
            "temp": {
                "type": "float",
                "description": "The temperature in the city",
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [26]:
set_weather_description = { "type": "function", "name": "set_weather", "description": "Add an entry to the weather database", "parameters": { "type": "object", "properties": { "city": { "type": "string", "description": "The city to be added to the weather database", }, "temp": { "type": "number", "description": "The temperature in the city", } }, "required": ["city", "temp"], "additionalProperties": False } }

In [27]:
tools.add_tool(set_weather,set_weather_description)

In [28]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data from the weather DB',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Check the weather in the city in question'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Add an entry to the weather database',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The city to be added to the weather database'},
    'temp': {'type': 'number', 'description': 'The temperature in the city'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [29]:
chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)
chat.run()

You: What's the temperature in Tokyo?


You: add this info to the weather database


You: stop


Chat ended.


In [30]:
!pip install fastmcp

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 38.9 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11103 sha256=d2182e2fd585560f5857a4f3d2ea11022fe1721dc3d277b903fc8abddae4f0e4
  Stored in directory: /home/codespace/.cache/pip/wheels/e0/e8/fc/8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [31]:
pip show fastmcp

Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [38]:
!pip install nest_asyncio

/usr/local/python/3.12.1/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=3063) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [39]:
import nest_asyncio
nest_asyncio.apply()

In [42]:
import asyncio

task = asyncio.create_task(mcp.run())


RuntimeError: Already running asyncio in this thread

In [45]:
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
    
await mcp.run_async()

╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯

AttributeError: 'OutStream' object has no attribute 'buffer'

In [46]:
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}

NameError: name 'true' is not defined